In [11]:
# Importing
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("sample_reviews.tsv", delimiter="\t", on_bad_lines="skip")

In [12]:
# Printing the Sample Reviews dataset
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,2975964,R1NBG94582SJE2,B00I01JQJM,860486164,GoPro Rechargeable Battery 2.0 (HERO3/HERO3+ o...,Camera,5,0,0,N,Y,Five Stars,ok,2015-08-31
1,US,23526356,R273DCA6Y0H9V7,B00TCO0ZAA,292641483,Professional 58mm Center Pinch Lens Cap for CA...,Camera,5,0,0,N,Y,Love it!!!,"Perfect, even sturdier than the original!",2015-08-31
2,US,52764145,RQVOXO7WUOFK6,B00B7733E0,75825744,Spy Tec Z12 Motion Activated Intelligent Secur...,Camera,2,1,1,N,Y,Another Motion Detect Fail,"If the words, &#34;Cheap Chinese Junk&#34; com...",2015-08-31
3,US,47348933,R1KWKSF21PO6HO,B006ZN4U34,789352955,"Celestron UpClose G2 10x25 Monocular, Black (7...",Camera,5,0,0,N,Y,Exactly what I wanted and expected.,Exactly what I wanted and expected. Perfect fo...,2015-08-31
4,US,33680700,R38H3UO1J190GI,B00HUEBGMU,19067902,Vidpro XM-L Wired Lavalier microphone - 20' Au...,Camera,5,1,1,N,Y,Good mic at a Good Price...Not Canon Though.,I will look past the fact that they tricked me...,2015-08-31


In [13]:
# Keep only the necessary columns
df = df[["customer_id", "review_id", "product_id", "product_category", "star_rating",
         "verified_purchase", "review_headline", "review_body", "review_date"]]


In [14]:
# Fill missing values in Review Body
df["review_body"] = df["review_body"].fillna("No Review")

In [17]:
# Convert Number Columns to Integers
df["customer_id"] = df["customer_id"].astype(int)
df["star_rating"] = df["star_rating"].astype(int)

# Convert Word Columns to Strings
df["review_headline"] = df["review_headline"].astype(str)
df["review_body"] = df["review_body"].astype(str)
df["product_category"] = df["product_category"].astype(str)


In [19]:
# Convert review date to Datetime
df["review_date"] = pd.to_datetime(df["review_date"])

# Set Vine and Verified Purchase to binary
df["verified_purchase"] = df["verified_purchase"].map({"Y": 1, "N": 0})


In [20]:
# Check the data types for each column
print(df.dtypes)

customer_id                   int32
review_id                    object
product_id                   object
product_category             object
star_rating                   int32
verified_purchase             int64
review_headline              object
review_body                  object
review_date          datetime64[ns]
dtype: object


In [21]:
# Spam condition 1: Spam words

# Define refined spam words
spam_related_words = [
    "scam", "fake", "bot", "fraudulent", "ripoff", "misleading",
    "counterfeit", "false", "bait", "switch", "waste", "useless"
]

# Flag unverified reviews containing spam-related words
def mark_word_spam_unverified(df, spam_related_words):
    df["contains_spam_words"] = df.apply(
        lambda row: 1 if (row["verified_purchase"] == 0 and any(word in row["review_body"].lower() for word in spam_related_words)) else 0,
        axis=1
    )
    return df

# Apply the refined function
df = mark_word_spam_unverified(df, spam_related_words)

# Check flagged review counts
print(df["contains_spam_words"].value_counts())

# Preview flagged spam reviews
df[df["contains_spam_words"] == 1].head(10)


contains_spam_words
0    9850
1     150
Name: count, dtype: int64


,customer_id,review_id,product_id,product_category,star_rating,verified_purchase,review_headline,review_body,review_date,contains_spam_words
53,42006015,R29EP0HGTKZVFG,B00WK55BQ6,Camera,5,0,Excellent IP/Wifi camera - great video quality...,"The Tenvis D200 (aka Tenvis TH661, Deecam D200...",2015-08-31,1
164,20666875,R1WS7VBVI8NH1Z,B00P8XG832,Camera,3,0,not bad not good just meh.,At best it can be consider a clone of the SJCA...,2015-08-31,1
240,7104987,R39NZO2II4CWTV,B00DUNYW7G,Camera,1,0,:(,"Very, VERY disappointed in this film. The &#34...",2015-08-31,1
250,52152582,R2N7OPDJI6E0WM,B00HLDFNKQ,Camera,2,0,A Great Leap Backward,At first glance the 340HS seems to be a techni...,2015-08-31,1
290,42006015,R38IEEIVJD23IR,B013OL3ANK,Camera,5,0,Excellent IP/Wifi camera - great video quality...,"The Deecam D200 (aka Tenvis TH661, Tenvis D200...",2015-08-31,1
301,49523782,R2GC63EU3MBG0R,B0116Q3U4Y,Camera,5,0,"(video test and demo) Works awesome, simple to...",[[VIDEOID:902e8616169ea6ef371d58868ce0a0e]] ✏️...,2015-08-31,1
333,46880653,R1S1HU9D43S0NG,B00H3Q8QB4,Camera,5,0,Foscam wirelss cam,Very nice Camera and the technical support was...,2015-08-31,1
527,20447137,R19SNJ3ZL7DDTA,B011M2HS1S,Camera,3,0,"A good start, but a work in progress ......","Looking for an exterior IP camera, I was impre...",2015-08-31,1
543,38173716,R2ERAFW5QK4PG4,B00X6BHOIQ,Camera,1,0,who runs a business like this?,It's so obvious that these are all fake review...,2015-08-31,1
573,50232752,R31VUZD3QT6YO9,B00WBJGUA2,Camera,4,0,"High quality product, great image, somewhat us...",I've had this device for a few weeks now and I...,2015-08-31,1


In [22]:
# Spam condition 2: Too few words

# Flag reviews with 5 or fewer words
df["short_review"] = df["review_body"].apply(lambda x: 1 if len(x.split()) <= 5 else 0)

# Mark spam to short reviews and spam_keyword reviews if they're also unverified
df["words_spam"] = df.apply(lambda row: 1 if (
    (row["short_review"] == 1 and row["verified_purchase"] == 0) or  # Must be Short & Unverified
    (row["short_review"] == 1 and any(word in row["review_body"].lower() for word in spam_related_words))  # Must be Short & Spam Words
) else 0, axis=1)

# Check the current spam count
print(df["words_spam"].value_counts())


words_spam
0    9966
1      34
Name: count, dtype: int64


In [23]:
# Spam condition 3: Too many reviews from same customer being sent too soon

# Sort reviews by customer_id and review_date
df = df.sort_values(by=["customer_id", "review_date"])

# Compute the difference in days between consecutive reviews from the same customer
df["same_day_review"] = df.groupby("customer_id")["review_date"].diff().dt.days

# Flag customers who post multiple reviews on the same day
df["fast_reviews"] = df["same_day_review"].apply(lambda x: 1 if x is not None and x == 0 else 0)

# Count how many fast reviews each customer posts per day
fast_reviews_per_day = df[df["fast_reviews"] == 1].groupby(["customer_id", "review_date"]).size().reset_index(name="fast_review_count")

# Flag customers who posted 10 or more fast reviews in a single day
spam_fast_customers = fast_reviews_per_day[fast_reviews_per_day["fast_review_count"] >= 10]["customer_id"].unique()

# Add a new spam column for fast reviewers
df["spam_fast_reviews"] = df["customer_id"].apply(lambda x: 1 if x in spam_fast_customers else 0)

# Check how many customers were flagged
print(df["spam_fast_reviews"].value_counts())


spam_fast_reviews
0    9892
1     108
Name: count, dtype: int64


In [24]:
# Spam condition 4: Repetitive Reviews by the Same Customer

# Set similarity threshold
SIMILARITY_THRESHOLD = 0.90

# Detect identical reviews function
def detect_similar_reviews(df, threshold=SIMILARITY_THRESHOLD):
    similar_reviews = []

    # Group reviews by customer_id
    for customer, reviews in df.groupby("customer_id")["review_body"]:
        if len(reviews) > 1:  # Customer must have multiple reviews
            vectorizer = TfidfVectorizer(stop_words="english")  # TF-IDF Vectorizer
            tfidf_matrix = vectorizer.fit_transform(reviews)  # Transform reviews into vectors

            # Compute cosine similarity for all reviews of the same customer
            similarity_matrix = cosine_similarity(tfidf_matrix)

            # Compare each review with others
            for i in range(len(reviews)):
                for j in range(i + 1, len(reviews)):  # Skip redundant comparisons
                    if similarity_matrix[i, j] >= threshold:
                        similar_reviews.append((customer, reviews.iloc[i], reviews.iloc[j], similarity_matrix[i, j]))

    return similar_reviews

# Run on dataset
similar_review_results = detect_similar_reviews(df)

# Convert results to a DataFrame
similar_review_df = pd.DataFrame(similar_review_results, columns=["customer_id", "review_1", "review_2", "similarity_score"])

df["repetitive_review_spam"] = df.apply(
    lambda row: 1 if (row["customer_id"] in similar_review_df["customer_id"].values and 
                      row["review_body"] in set(similar_review_df["review_1"]).union(set(similar_review_df["review_2"]))) else 0,
    axis=1
)

# Show the first few results
similar_review_df.head()

,customer_id,review_1,review_2,similarity_score
0,109766,Item as described,Item as described,1.000000
1,111870,Good,Good,1.000000
2,123943,The video not clear.<br />The sound is bad and...,The video not clear.<br />The sound is bad and...,0.926773
3,137226,great!,great!,1.000000
4,154445,great,great,1.000000


In [27]:
# Spam condition 5: Reviews with URLs

# Regex pattern for detecting URLs
url_pattern = r"https?://\S+|www\.\S+|\b[a-zA-Z0-9.-]+\.(com|net|org|edu|gov|co|io|me|us|uk|ca|de|jp|fr|au|in)\b"

# Function to detect URLs in review_body
def mark_link_spam(df):
    df["contains_link_spam"] = df["review_body"].apply(lambda x: 1 if re.search(url_pattern, x) else 0)
    return df

# Apply the function
df = mark_link_spam(df)

# Count the # of URL reviews flagged
print(df["contains_link_spam"].value_counts())

# Preview flagged spam reviews
df[df["contains_link_spam"] == 1].head(10)


contains_link_spam
0    9925
1      75
Name: count, dtype: int64


,customer_id,review_id,product_id,product_category,star_rating,verified_purchase,review_headline,review_body,review_date,contains_spam_words,short_review,words_spam,same_day_review,fast_reviews,spam_fast_reviews,repetitive_review_spam,contains_link_spam,spam
5477,621415,R39LKA3HL7SIEG,B00RKNN322,Camera,2,0,DISAPPOINTED. CAN GET BETTER CAMERA QUALITY FO...,[[VIDEOID:a1f385b2852cff6485efc8a7826e782]] Pl...,2015-08-28,0,0,0,NaN,0,0,0,1,1
4858,712156,R3PE9VFR3BMUOL,B00MYHUVWE,Camera,5,0,This slider is dope!!!,[[VIDEOID:ef330d76bb7f6ab530d3909f8f557820]] I...,2015-08-29,0,0,0,NaN,0,0,0,1,1
7256,781699,RFTVS22U98YLY,B012N05T6Y,Camera,5,0,Great wireless IP camera.,Dbpower is a great company and i love all the ...,2015-08-27,0,0,0,NaN,0,0,0,1,1
1806,954022,R3PTSJUDJTHG1M,B00S1N1ESU,Camera,5,1,This little camera works great! VERY cool,"This little camera works great! VERY cool, and...",2015-08-31,0,0,0,NaN,0,0,0,1,1
7202,1572275,RLI1RC5FFQJVP,B00BIY0DFY,Camera,2,1,tried to get in touch with company know one an...,tried to get in touch with company know one a...,2015-08-27,0,0,0,0.0,1,0,0,1,1
535,2784262,R1QCPB3M9JHUHR,B00AWLRLYK,Camera,3,1,A good but not great item,If you want to read my full review with some p...,2015-08-31,0,0,0,NaN,0,0,0,1,1
8666,5353624,R2PJ0C08306TCF,B00PQ4WXKA,Camera,1,1,One Star,Crapy product. Save your money and get a proma...,2015-08-26,0,0,0,NaN,0,0,0,1,1
2408,8906086,R223GUFUVH0NHN,B0131PHDQY,Camera,2,0,Too many problems ...,I had a problem with motion detection feature ...,2015-08-30,1,0,0,NaN,0,0,0,1,1
181,11213801,R1VR1OVGA2L7OC,B003Y73Q3Y,Camera,1,0,We are very disappointed with Defender's failu...,"We purchased a Defender brand DVR on March 3, ...",2015-08-31,0,0,0,NaN,0,0,1,1,1
604,11213801,R1UDKKQDO6I03Q,B00IB6I6SI,Camera,1,0,We are very disappointed with Defender's failu...,"We purchased a Defender brand DVR on March 3, ...",2015-08-31,0,0,0,0.0,1,0,1,1,1


In [28]:
# Combine all spam conditions into one "spam" column
df["spam"] = df[[
    "contains_spam_words",
    "words_spam",
    "spam_fast_reviews",
    "repetitive_review_spam",
    "contains_link_spam"
]].max(axis=1)

# Check the final spam count
print(df["spam"].value_counts())

# Check Dataframe
df[df["spam"] == 1].head(10)

spam
0    9250
1     750
Name: count, dtype: int64


,customer_id,review_id,product_id,product_category,star_rating,verified_purchase,review_headline,review_body,review_date,contains_spam_words,short_review,words_spam,same_day_review,fast_reviews,spam_fast_reviews,repetitive_review_spam,contains_link_spam,spam
3025,92693,R3QHR1B0GS4TB,B00M8VA1C6,Camera,1,0,CHEAP MATERIAL NO GOOD,VERY CHEAP MATERIAL ... BATTERIES RUN LOW FAST...,2015-08-30,1,0,0,NaN,0,0,0,0,1
6490,109766,R154OIZ7RBZICD,B007GAIQK8,Camera,5,1,Five Stars,Item as described,2015-08-27,0,1,0,NaN,0,0,1,0,1
6955,109766,R1HDG9JISUQSAH,B00EBPNCTQ,Camera,5,1,Five Stars,Item as described,2015-08-27,0,1,0,0.0,1,0,1,0,1
1350,111870,RCV4A3FG51GZ8,B002JIN1IO,Camera,5,1,Five Stars,Good,2015-08-31,0,1,0,NaN,0,0,1,0,1
1522,111870,R10ZFX8O1XU4NN,B00TE8XMMM,Camera,5,1,Five Stars,Good,2015-08-31,0,1,0,0.0,1,0,1,0,1
8253,116729,R3IAFX1N1ZCQR4,B010PM56G0,Camera,5,0,Five Stars,Everything we needed!,2015-08-26,0,1,1,NaN,0,0,0,0,1
6018,119502,R47EN8PNTWLOC,B00HNJWUBI,Camera,4,0,For that record-of-the-moment shot,For a basic snap camera the W830 is fine. Lots...,2015-08-28,1,0,0,NaN,0,0,0,0,1
6820,123943,R1PJMCDR53CD0H,B004P8K24W,Camera,1,1,The sound is bad and nosy,The video not clear.<br />The sound is bad and...,2015-08-27,0,0,0,NaN,0,0,1,0,1
7366,123943,R11MYMOQDT0MU4,B00Y9XC2A0,Camera,1,1,The sound is bad and nosy,The video not clear.<br />The sound is bad and...,2015-08-27,0,0,0,0.0,1,0,1,0,1
4718,125118,R24WD2P80TNLAF,B00RKNMLRU,Camera,5,0,Five Stars,As expected!,2015-08-29,0,1,1,NaN,0,0,0,0,1
